□ [nn-morse](https://github.com/pd0wm/nn-morse)で学習したモデル(onnx形式)からonnxruntimeだけで推論 (part2: SatNOGSのオーディオファイルに対応)

■ GoogleDriveに接続

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


■ 環境構築

In [2]:
!cp drive/MyDrive/MORSE/新models-8/main8.py main.py
!cp drive/MyDrive/MORSE/新models-8/my_morse.py morse.py
!cp "drive/MyDrive/MORSE/新models-8/nn-morse.onnx" nn-morse.onnx

!cp drive/MyDrive/MORSE/hsu-sat1.ogg .

■ onnxruntimeから推論

In [3]:
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.8 MB/s eta 0:00:00


3. oggファイルの読み込み

In [4]:
!pip install pydub

In [5]:
import numpy as np
from scipy import signal
from pydub import AudioSegment

from main import num_tags, prediction_to_str
from morse import ALPHABET, SAMPLE_FREQ, get_spectrogram

audio = AudioSegment.from_file("hsu-sat1.ogg", format="ogg")
data = np.array(audio.get_array_of_samples())
rate = audio.frame_rate
#print (type(data))
#print (rate)

# Resample and rescale
length = len(data) / rate
new_length = int(length * SAMPLE_FREQ)

data = signal.resample(data, new_length)
data = data.astype(np.float32)
data /= np.max(np.abs(data))

# Create spectrogram
spec = get_spectrogram(data)
spec_orig = spec.copy()
spectrogram_size = spec.shape[0]

- numpyでspecを加工するよう修正

In [6]:
spec = spec.transpose(1,0)
print (spec)
spec = np.expand_dims(spec, axis=0)
print (spec)

[[9.3274984e-06 3.4138023e-07 2.5348359e-07 ... 2.0061675e-06
  2.4245978e-06 4.5530109e-05]
 [2.4899841e-13 3.3079494e-11 1.5759806e-12 ... 5.8261429e-07
  7.8037226e-07 2.0087686e-05]
 [1.3343058e-11 2.3198521e-09 1.1770801e-09 ... 3.9835231e-07
  5.4680731e-07 1.5002691e-05]
 ...
 [8.4079750e-04 3.7355728e-03 3.4301504e-03 ... 4.6602762e-03
  4.9454304e-03 5.4244383e-04]
 [2.9601466e-05 1.9074654e-03 6.4556231e-03 ... 2.6051138e-04
  7.1842493e-03 6.0188879e-05]
 [4.2016557e-04 4.9430588e-03 8.4554097e-03 ... 6.7210043e-05
  1.1158641e-03 2.6159626e-03]]
[[[9.3274984e-06 3.4138023e-07 2.5348359e-07 ... 2.0061675e-06
   2.4245978e-06 4.5530109e-05]
  [2.4899841e-13 3.3079494e-11 1.5759806e-12 ... 5.8261429e-07
   7.8037226e-07 2.0087686e-05]
  [1.3343058e-11 2.3198521e-09 1.1770801e-09 ... 3.9835231e-07
   5.4680731e-07 1.5002691e-05]
  ...
  [8.4079750e-04 3.7355728e-03 3.4301504e-03 ... 4.6602762e-03
   4.9454304e-03 5.4244383e-04]
  [2.9601466e-05 1.9074654e-03 6.4556231e-03 ... 2

4. 推論

In [7]:
import onnxruntime as ort

sess = ort.InferenceSession('nn-morse.onnx')
output = sess.run(['output'], {'input': spec})

In [8]:
print ("◇output")
print (output)

output1 = np.array(output[0][0])
print ("◆y_pred⇐output1")
print (output1)

y_pred_l = np.exp(output1.tolist())
print ("◆y_pred_l")
print (y_pred_l)


◇output
[array([[[ 0.0000000e+00, -2.1061577e+01, -2.0310669e+01, ...,
         -2.4417110e+01, -2.4160887e+01, -2.4291821e+01],
        [ 0.0000000e+00, -3.8936188e+01, -3.3078648e+01, ...,
         -4.3582195e+01, -4.2318993e+01, -3.7926723e+01],
        [ 0.0000000e+00, -3.8175549e+01, -3.1148102e+01, ...,
         -3.9882454e+01, -3.8202057e+01, -3.4655582e+01],
        ...,
        [-1.0793343e-03, -1.0876624e+01, -1.1870118e+01, ...,
         -1.2586566e+01, -1.1282993e+01, -1.3150455e+01],
        [-8.9986355e-04, -1.2185913e+01, -1.1785722e+01, ...,
         -1.2427601e+01, -1.2001192e+01, -1.2934656e+01],
        [-6.6992450e-03, -9.6875887e+00, -9.5118628e+00, ...,
         -1.0002978e+01, -9.1782799e+00, -1.0220168e+01]]], dtype=float32)]
◆y_pred⇐output1
[[ 0.0000000e+00 -2.1061577e+01 -2.0310669e+01 ... -2.4417110e+01
  -2.4160887e+01 -2.4291821e+01]
 [ 0.0000000e+00 -3.8936188e+01 -3.3078648e+01 ... -4.3582195e+01
  -4.2318993e+01 -3.7926723e+01]
 [ 0.0000000e+00 -3.817554

5. 文字列に変換

In [9]:
# Convert prediction into string
# TODO: proper beam search
m = np.argmax(output1, 1)
print ("◆m")
print (m)

print ("変換")
print(prediction_to_str(m))

◆m
[0 0 0 ... 0 0 0]
変換
H4.18V0.08A8.14DEEEEETTTTEE  0JS1YHSHSUSAT104.19V0.10A8.46DEEEEETETTTE 9I   U  J4 V1 O: EDI       ISW               1:   I   


6. 結果比較. wavファイルと同じになった!

```
H4.18V0.08A8.14DEEEEETTTTEE  0JS1YHSHSUSAT104.19V0.10A8.46DEEEEETETTTE 9I   U  J4 V1 O: EDI       ISW               1:   I                                                IA            M  
```